In [1]:
from imutils.contours import sort_contours
import numpy as np
import imutils
import cv2
import pickle

In [2]:
loaded_model = pickle.load(open('Pickle_Model.pkl', 'rb'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
image = cv2.imread('K-K-K.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
dst = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)

In [4]:
edged = cv2.Canny(dst, 30, 150)
cnts,h = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = sort_contours(cnts, method="left-to-right")[0]


In [5]:
chars = []

In [6]:
for c in cnts:

    (x, y, w, h) = cv2.boundingRect(c)


    if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):

        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(roi, 150, 255,
            cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        (tH, tW) = thresh.shape

        if tW > tH:
            thresh = imutils.resize(thresh, width=28)
        else:
            thresh = imutils.resize(thresh, height=28)
            
        (tH, tW) = thresh.shape
        dX = int(max(0, 28 - tW) / 2.0)
        dY = int(max(0, 28 - tH) / 2.0)

        padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
            left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
            value=(0, 0, 0))
        padded = cv2.resize(padded, (28, 28))


        padded = padded.astype("float32") / 255.0
        padded = np.expand_dims(padded, axis=-1)

        chars.append((padded, (x, y, w, h)))

In [7]:
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")
preds = loaded_model.predict(chars)
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [8]:
lst=list()
word=""
for (pred, (x, y, w, h)) in zip(preds, boxes):

    i = np.argmax(pred)
    prob = pred[i]
    label = labelNames[i]

    print("[INFO] {} - {:.2f}%".format(label, prob * 100))
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    word+=label
    
    cv2.putText(image, label, (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    
    

    cv2.imshow("Image", image)
    cv2.waitKey(0)

[INFO] K - 55.49%
[INFO] E - 0.40%
[INFO] V - 11.73%
[INFO] I - 0.32%
[INFO] N - 0.54%


In [9]:
print(word)

KEVIN
